Exercise: USA Real Estate Price Prediction
Dataset: USA Real Estate Dataset (200k+ entries)
Link: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
This dataset contains real estate listings across the USA with features like location (latitude, longitude, city, state), property characteristics (bedrooms, bathrooms, square footage), and prices.

In [2]:
import sys

Installing the packages

In [6]:
!pip install numpy pandas scikit-learn
!pip install matplotlib seaborn
!pip install kaggle
!pip install packaging


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 49.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 51.1 MB/s eta 0:00:0031m52.9 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached contourpy-1.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 49.3 MB/s eta 0:00:00
Using cached contourpy-1.3.3-cp312-cp312-macosx_11_0_arm64.whl (273 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.7 MB/s eta 0:00:00
Using cached kiwisolver-1.4.9-cp312-cp

In [8]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

import numpy as np
import pandas as pd
import sklearn
from packaging import version

Get the Data:

Download the Data and creating a file csv containing it

In [11]:
!kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset
!unzip -o usa-real-estate-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
License(s): other
usa-real-estate-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  usa-real-estate-dataset.zip
  inflating: realtor-data.zip.csv    


In [10]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import warnings

In [13]:
df = pd.read_csv('realtor-data.csv')
print(f"Dataset Shape:{df.shape}")
print(f"\nColumns : {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

Dataset Shape:(2226382, 12)

Columns : ['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street', 'city', 'state', 'zip_code', 'house_size', 'prev_sold_date']

First few rows:
   brokered_by    status     price  bed  bath  acre_lot     street  \
0     103378.0  for_sale  105000.0  3.0   2.0      0.12  1962661.0   
1      52707.0  for_sale   80000.0  4.0   2.0      0.08  1902874.0   
2     103379.0  for_sale   67000.0  2.0   1.0      0.15  1404990.0   
3      31239.0  for_sale  145000.0  4.0   2.0      0.10  1947675.0   
4      34632.0  for_sale   65000.0  6.0   2.0      0.05   331151.0   

         city        state  zip_code  house_size prev_sold_date  
0    Adjuntas  Puerto Rico     601.0       920.0            NaN  
1    Adjuntas  Puerto Rico     601.0      1527.0            NaN  
2  Juana Diaz  Puerto Rico     795.0       748.0            NaN  
3       Ponce  Puerto Rico     731.0      1800.0            NaN  
4    Mayaguez  Puerto Rico     680.0         NaN            

In [15]:
#Let's check for any missing values
print(f"Missing Values:")
print(df.isnull().sum())

Missing Values:
brokered_by         4533
status                 0
price               1541
bed               481317
bath              511771
acre_lot          325589
street             10866
city                1407
state                  8
zip_code             299
house_size        568484
prev_sold_date    734297
dtype: int64


In [16]:
#Since there is some missing data (especially in the price that is the value we want to predict)
df_clean = df.dropna(subset=['price'])
#And now let's select our features from the dataset
feature_columns = ['bed', 'bath', 'acre_lot', 'house_size']
dl_clean = df_clean.dropna(subset=feature_columns)

In [17]:
X = df_clean[feature_columns]
y = df_clean['price']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nTarget statistics:")
print(y.describe())

Features shape: (2224841, 4)
Target shape: (2224841,)

Target statistics:
count    2.224841e+06
mean     5.241955e+05
std      2.138893e+06
min      0.000000e+00
25%      1.650000e+05
50%      3.250000e+05
75%      5.500000e+05
max      2.147484e+09
Name: price, dtype: float64


In [18]:
# Use only first 10,000 samples as specified
X_sample = X.head(10000)
y_sample = y.head(10000)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 8000
Test set size: 2000
